In [ ]:
import numpy as np
import theano
import theano.tensor as T
import itertools
import time

def binFunc(v):
    return [(v[0] and v[1] and v[4]) or (v[4] and v[6] and v[2] and v[7]) or (v[1] and v[3] and v[4]) or (v[2] and v[5] and v[6] and v[7])]

# hyperparameter
learningRate = .1
learningSteps = 50000

# input dataset

X = np.array([i for i in itertools.product([0,1], repeat=8)])
    
# output dataset
Y = np.array([binFunc(v) for v in X])
# seed random numbers to make calculation
# deterministic (just a good practice)
np.random.seed(1)

# initialize weights randomly with mean 0
w1 = theano.shared(2 * np.random.random((X.shape[1], 10)) - 1, 'w1')
w2 = theano.shared(2 * np.random.random((10, 1)) - 1, 'w2')

# define sigmoid and error function
x = T.matrix('x', dtype=theano.config.floatX)
y = T.matrix('y', dtype=theano.config.floatX)

ff_i_h = 1 / (1 + T.exp(-T.dot(x, w1)))
ff = 1 / (1 + T.exp(-T.dot(ff_i_h, w2)))

err =  T.sum(ff - y)
err = T.sum(T.pow(ff - y, 2))/(2 * X.shape[0])
w1_grad = T.grad(err, w1)
w2_grad = T.grad(err, w2)

predict = theano.function([x], ff, allow_input_downcast=True)
train = theano.function([x, y], err, updates=[(w1, w1 - learningRate * w1_grad), (w2, w2 - learningRate * w2_grad)], allow_input_downcast=True)

# learning process
t0 = time.time()
for iter in xrange(learningSteps):
    train(X, Y)
print u'finished'
t1 = time.time()

res = []
for g, v in zip(predict(X), Y):
    if (v == 1 and g >= .5) or (v == 0 and g < .5):
        res.append(1)
    else:
        res.append(0)
res = np.array(res)

print(u'win rate: {}. Took: {}s. device: {}'.format(np.where(res > 0)[0].__len__()/float(X.shape[0]), t1-t0, theano.config.device))
